<a href="https://colab.research.google.com/github/khushee-g/EEG/blob/main/Band_pass___Final_Model_Sigmoid_function_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.axes as ax
from matplotlib.animation import FuncAnimation

In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

top_features = ['COH.A.delta.a.FP1.b.FP2', 'COH.A.delta.f.F4.h.T3',
                   'AB.C.alpha.q.T6', 'AB.B.theta.c.F7']
data1 = pd.read_csv('/content/numericBehaviorWithAgeOnly.csv')

Y = data1['specific_disorder']
X = data1[top_features]

Y = data1['specific_disorder'] == 5

X_with_const = sm.add_constant(X)
logit_model = sm.Logit(Y, X_with_const)
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.648650
         Iterations 5


# OVERALL MODEL ACCURACY

In [4]:
import random
random.seed(16)
# import the necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

df = pd.read_csv('/content/numericBehaviorWithAgeOnly.csv')


X = df.drop(columns=['specific_disorder'])
X = X[[col for col in X.columns if col in top_features]]

y = df['specific_disorder']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

clf = LogisticRegression(random_state=0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(y_pred)

acc = accuracy_score(y_test, y_pred)
print("Logistic Regression model accuracy (in %):", acc * 100)


[5 5 4 4 5 5 4 5 5 4 4 5 5 5 5 5 5 5 5 5 4 4 4 4 4 4 4 4 5 4 5 4 4 5 5 4 4
 5]
Logistic Regression model accuracy (in %): 68.42105263157895


# Muse Integration

In [9]:
user_data = pd.read_csv('/content/Khushee mindMonitor_2024-11-15--21-52-00.csv')
data1 = pd.read_csv('/content/numericBehaviorWithAgeOnly.csv')


<ipython-input-9-68862582fc80>:1: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  user_data = pd.read_csv('/content/Khushee mindMonitor_2024-11-15--21-52-00.csv')


In [10]:
import numpy as np
from scipy.signal import butter, filtfilt
import pandas as pd

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

fs = 256
lowcut = 0.5
highcut = 4

data = user_data
num_nan_raw_AF8 = data['RAW_AF8'].isnull().sum()
num_nan_raw_AF7 = data['RAW_AF7'].isnull().sum()
num_nan_raw_TP9 = data['RAW_TP9'].isnull().sum()
num_nan_raw_TP10 = data['RAW_TP10'].isnull().sum()

if num_nan_raw_AF8 > 0:
    data['RAW_AF8'] = data['RAW_AF8'].interpolate(method='linear')

if num_nan_raw_AF7 > 0:
    data['RAW_AF7'] = data['RAW_AF7'].interpolate(method='linear')

if num_nan_raw_TP9 > 0:
    data['RAW_TP9'] = data['RAW_TP9'].interpolate(method='linear')

if num_nan_raw_TP10 > 0:
    data['RAW_TP10'] = data['RAW_TP10'].interpolate(method='linear')


filtered_delta_AF8 = bandpass_filter(data['RAW_AF8'].values, lowcut, highcut, fs, order=5)
filtered_delta_AF7 = bandpass_filter(data['RAW_AF7'].values, lowcut, highcut, fs, order=5)
filtered_delta_TP9 = bandpass_filter(data['RAW_TP9'].values, lowcut, highcut, fs, order=5)
filtered_delta_TP10 = bandpass_filter(data['RAW_TP10'].values, lowcut, highcut, fs, order=5)



fs = 256
lowcut = 4
highcut = 8

num_nan_raw_AF7 = data['RAW_AF7'].isnull().sum()

if num_nan_raw_TP10 > 0:
    data['RAW_AF7'] = data['RAW_AF7'].interpolate(method='linear')


filtered_theta_AF7 = bandpass_filter(data['RAW_AF7'].values, lowcut, highcut, fs, order=5)



In [8]:
#Delta AF7-AF8 coherence - corresponds to COH.A.delta.a.FP1.b.FP2

from scipy import signal


channel_1 = filtered_delta_AF8
channel_2 = filtered_delta_AF7

channel_1_mean = np.nanmean(channel_1)
channel_2_mean = np.nanmean(channel_2)

channel_1 = np.nan_to_num(channel_1, nan=np.nanmean(channel_1))
channel_2 = np.nan_to_num(channel_2, nan=np.nanmean(channel_2))

fs = 256
f, Cxy = signal.coherence(channel_1, channel_2, fs=fs, nperseg=1024)  # fs is sampling frequency

coherence_data = pd.DataFrame({'Frequency': f, 'Coherence DeltaAF7DeltaAF8': Cxy})

coherence_AF8_AF7 = coherence_data['Coherence DeltaAF7DeltaAF8'].mean()
coherence_AF8_AF7 *=100
print(coherence_AF8_AF7)




#Delta AF8-TP9 coherence - corresponds to COH.A.delta.f.F4.h.T3

from scipy import signal


channel_1 = filtered_delta_AF8
channel_1 = filtered_delta_TP9

channel_1_mean = np.nanmean(channel_1)
channel_2_mean = np.nanmean(channel_2)

channel_1 = np.nan_to_num(channel_1, nan=np.nanmean(channel_1))
channel_2 = np.nan_to_num(channel_2, nan=np.nanmean(channel_2))

fs = 256
f, Cxy = signal.coherence(channel_1, channel_2, fs=fs, nperseg=1024)  # fs is sampling frequency

coherence_data = pd.DataFrame({'Frequency': f, 'Coherence DeltaAF8DeltaTP9': Cxy})

coherence_AF8_TP9 = coherence_data['Coherence DeltaAF8DeltaTP9'].mean()
coherence_AF8_TP9 *=100
print(coherence_AF8_TP9)

50.97487654935996
51.55583751588699


In [11]:
#COH.A.delta.a.FP1.b.FP2
X = data1['COH.A.delta.a.FP1.b.FP2']

Y = data1['specific_disorder'].apply(lambda x: 1 if x == 5 else -1)

X_with_const = sm.add_constant(X)
logit_model = sm.OLS(Y, X_with_const)
result = logit_model.fit()


prediction_data = pd.DataFrame({'const': [1], 'COH.A.delta.a.FP1.b.FP2': [coherence_AF8_AF7]})

pred = result.get_prediction(prediction_data)


#risk1 = pred.predicted_mean[0]
risk1 = 0.0246*(coherence_AF8_AF7) - 1.7527
print(risk1)
risk1_sigmoid = p = 1 / (1 + np.exp(-risk1))
print(risk1_sigmoid)

#Note: significant difference between equation-based prediction and pred.predicted_mean[0]

-0.4987180368857449
0.37784198215032


In [12]:

Y = data1['specific_disorder'] == 5
X = data1['COH.A.delta.f.F4.h.T3']

Y = data1['specific_disorder'].apply(lambda x: 1 if x == 5 else -1)

X_with_const = sm.add_constant(X)
logit_model = sm.OLS(Y, X_with_const)
result = logit_model.fit()


prediction_data = pd.DataFrame({'const': [1], 'COH.A.delta.f.F4.h.T3': [coherence_AF8_TP9]})

pred = result.get_prediction(prediction_data)


#risk2 = pred.predicted_mean[0]
risk2 =  -0.0040*(coherence_AF8_TP9) + 0.1324
print(risk2)
risk2_sigmoid = p = 1 / (1 + np.exp(-risk2))
print(risk2_sigmoid)

-0.07382335006354798
0.48155253977105095


In [13]:
#AB.C.alpha.q.T6
Y = data1['specific_disorder'] == 5
X = data1['AB.C.alpha.q.T6']

Y = data1['specific_disorder'].apply(lambda x: 1 if x == 5 else -1)

X_with_const = sm.add_constant(X)
logit_model = sm.OLS(Y, X_with_const)
result = logit_model.fit()

Delta_TP10 = filtered_delta_TP10
Delta_TP10 = np.nan_to_num(channel_1, nan=np.nanmean(channel_1))
Delta_TP10 = Delta_TP10.mean()

prediction_data = pd.DataFrame({'const': [1], 'AB.C.alpha.q.T6': [Delta_TP10]})

pred = result.get_prediction(prediction_data)


#risk3 = pred.predicted_mean[0]
risk3 = 0.0097*data['Delta_TP10'].mean() - 0.1531
print(risk3)
risk3_sigmoid = p = 1 / (1 + np.exp(-risk3))
print(risk3_sigmoid)

-0.14750334114591296
0.4631908792513958


In [14]:
#AB.B.theta.c.F7
Y = data1['specific_disorder'] == 5
X = data1['AB.B.theta.c.F7']

Y = data1['specific_disorder'].apply(lambda x: 1 if x == 5 else -1)

X_with_const = sm.add_constant(X)
logit_model = sm.OLS(Y, X_with_const)
result = logit_model.fit()

Theta_AF7 = filtered_theta_AF7
Theta_AF7 = np.nan_to_num(channel_1, nan=np.nanmean(channel_1))
Theta_AF7 = Theta_AF7.mean()

prediction_data = pd.DataFrame({'const': [1], 'AB.B.theta.c.F7': [Theta_AF7]})

pred = result.get_prediction(prediction_data)


#risk4 = pred.predicted_mean[0]
risk4 = -0.0152 *data['Theta_AF7'].mean() + 0.2759
print(risk4)
risk4_sigmoid = p = 1 / (1 + np.exp(-risk4))
print(risk4_sigmoid)

0.27346845326856967
0.5679442062276707


In [15]:
if risk4_sigmoid < 0.5:
  risk4_sigmoid = -1*(0.5 - risk4_sigmoid)

if risk3_sigmoid < 0.5:
  risk3_sigmoid = -1*(0.5 - risk3_sigmoid)

if risk2_sigmoid < 0.5:
  risk2_sigmoid = -1*(0.5 - risk2_sigmoid)

if risk1_sigmoid < 0.5:
  risk1_sigmoid = -1*(0.5 - risk1_sigmoid)

if risk4_sigmoid > 0.5:
  risk4_sigmoid = (risk4_sigmoid-0.5)
if risk3_sigmoid > 0.5:
  risk3_sigmoid = (risk3_sigmoid-0.5)
if risk2_sigmoid > 0.5:
  risk2_sigmoid = (risk2_sigmoid-0.5)
if risk1_sigmoid > 0.5:
  risk1_sigmoid = (risk1_sigmoid-0.5)


risk_sigmoid_total = risk4_sigmoid + risk3_sigmoid + risk2_sigmoid + risk1_sigmoid

if risk_sigmoid_total<0:
  print("You are " + str(risk_sigmoid_total*-100) +"% less likely to have IAD")
if risk_sigmoid_total>0:
  print("You are " + str(risk_sigmoid_total*100) +"% more likely to have IAD")
if risk_sigmoid_total==0:
  print("Prediction cannot be made.")

#equation based: 25.56406586142109% less likely
#prediction fuction based: 15.022754660005367% more likely

You are 10.947039259956254% less likely to have IAD


In [16]:
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np

# Initialize the dictionary to store the odds ratios
odds_ratios_dict = {}

channel_num = 0

# Assuming top_features is defined somewhere before the loop
for i in top_features:
    print(f"channel {i}")

    # Rename the column for the current channel
    data1.rename(columns={i: f'channel_{channel_num}'}, inplace=True)
    print(data1[f'channel_{channel_num}'])

    # Prepare the X matrix (only the current channel)
    X = data1[[f'channel_{channel_num}']]

    # Convert 'specific_disorder' to a binary variable (assuming '5' is the target class)
    data1['specific_disorder_binary'] = (data1['specific_disorder'] == 5).astype(int)

    # Define the formula for logistic regression
    formula_str = f"specific_disorder_binary ~ channel_{channel_num}"  # Update formula with current channel
    print(formula_str)

    # Fit the logistic regression model
    log_reg = smf.logit(formula_str, data=data1).fit()
    print(log_reg.summary())

    # Calculate the odds ratios and their confidence intervals
    odds_ratios = pd.DataFrame(
        {
            "OR": log_reg.params,
            "Lower CI": log_reg.conf_int()[0],
            "Upper CI": log_reg.conf_int()[1],
        }
    )
    odds_ratios = np.exp(odds_ratios)  # Exponentiate to get odds ratios

    # Save each odds ratio as a separate variable in the dictionary
    odds_ratios_dict[f'channel_{channel_num}_odds_ratio'] = odds_ratios

    channel_num += 1
    print(f"ODDS RATIO for channel {i}:")
    print(odds_ratios)

# Optionally, you can print out all the stored odds ratios
print("\nAll Odds Ratios saved in the dictionary:")
for key, value in odds_ratios_dict.items():
    print(f"{key}:")
    print(value)


channel COH.A.delta.a.FP1.b.FP2
0      40.893243
1      35.672185
2      74.051341
3      90.006558
4      63.756704
         ...    
183    64.188787
184    59.095639
185    53.635964
186    80.930246
187    99.775084
Name: channel_0, Length: 188, dtype: float64
specific_disorder_binary ~ channel_0
Optimization terminated successfully.
         Current function value: 0.668620
         Iterations 5
                              Logit Regression Results                              
Dep. Variable:     specific_disorder_binary   No. Observations:                  188
Model:                                Logit   Df Residuals:                      186
Method:                                 MLE   Df Model:                            1
Date:                      Sun, 02 Feb 2025   Pseudo R-squ.:                 0.03531
Time:                              21:02:51   Log-Likelihood:                -125.70
converged:                             True   LL-Null:                       -130.30
Co